<a href="https://colab.research.google.com/github/fersebas/data_kdd_recopilation/blob/master/SCRAPING_PRICES_USING_SKYSCANNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 1. Funciones api skyscanner

In [0]:
def create_session(depart_date, arrival_date):   
  import requests
  url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/v1.0"

  payload = "inboundDate="+arrival_date+"&cabinClass=economy&children=0&infants=0&country=ES&currency=EUR&locale=es-ES&originPlace=MAD-sky&destinationPlace=LPA-sky&outboundDate="+depart_date+"&adults=1"
  headers = {
      'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
      'x-rapidapi-key': "c87701c514msh05e1981eb4b7ba3p106611jsn5b01d80662c5",
      'content-type': "application/x-www-form-urlencoded"
      }


  while True:
    response = requests.request("POST", url, data=payload, headers=headers)
    if response.status_code == 201:
      print("Sesion creada")
      print(response.text)
      break
  return response


In [11]:
r = create_session("2019-10-11", "2019-10-13")

Sesion creada
{}


In [0]:
def get_json(response):
  # SessionKey 	The Session key to identify the session.
  # 1. Query - A copy of the query which was submitted.
  # 2. Status -	The status of the session – ‘UpdatesPending’ or ‘UpdatesComplete’.
  # 3. Itineraries - A list of itineraries - see below for the itinerary object.
  # 4. Legs -	Details of the legs that make up the itineraries: airports, times, overall duration, stops and carrier ids.
  # 5. Segments - Details of the segments of each leg. Including the carrier (or marketing carrier) and the operating carrier.
  # 6. Carriers -	Details of the carriers.
  # 7. Agents -	Details of the agents who sell the tickets. Can be an airline or a travel agent.
  # 8. Places -	A list of all the places that appear in the itineraries.
  # 9. Currencies - A list of the currencies shown in the response.

  url = response.headers["Location"]
  session_id = url.split("/")[-1]
  import requests
  import json
  url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/pricing/uk2/v1.0/"+session_id

  querystring = {"sortType":"price","sortOrder":"asc","stops":"0","pageIndex":"0","pageSize":"10"}

  headers = {
      'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
      'x-rapidapi-key': "c87701c514msh05e1981eb4b7ba3p106611jsn5b01d80662c5"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json()

In [8]:
json_data = get_json(r)

NameError: ignored

In [9]:
json_data["Status"]

NameError: ignored

#**1. Query**

In [0]:
Query = pd.DataFrame.from_records([json_data['Query']])

In [0]:
Query

,Adults,CabinClass,Children,Country,Currency,DestinationPlace,GroupPricing,InboundDate,Infants,Locale,LocationSchema,OriginPlace,OutboundDate
0,1,economy,0,ES,EUR,13689,False,2019-10-14,0,es-ES,Default,13870,2019-10-11


# **2. Status**

In [0]:
Status = json_data['Status']

In [0]:
Status

'UpdatesPending'

# **3. Itineraries**

In [0]:
Itineraries = pd.DataFrame()
num_iteraries = len(json_data['Itineraries'])
for i in range(num_iteraries):
  new = pd.DataFrame(json_data['Itineraries'][i]['PricingOptions'])
  new["InboundLegId"] = json_data['Itineraries'][i]['InboundLegId']
  new["OutboundLegId"] = json_data['Itineraries'][i]['OutboundLegId']
  Itineraries = Itineraries.append(new)
Itineraries.reset_index(inplace=True)
Itineraries.head(2)

,index,Agents,DeeplinkUrl,Price,QuoteAgeInMinutes,InboundLegId,OutboundLegId
0,0,[4035342],http://partners.api.skyscanner.net/apiservices...,147.85,188,13689-1910131800--32680-0-13870-1910132150,13870-1910110705--32680-0-13689-1910110900
1,1,[3986805],http://partners.api.skyscanner.net/apiservices...,148.03,100,13689-1910131800--32680-0-13870-1910132150,13870-1910110705--32680-0-13689-1910110900


# **4. Legs**

In [0]:
Legs = pd.DataFrame()
num_legs = len(json_data['Legs'])
for i in range(num_legs):
  Legs = Legs.append(pd.DataFrame([json_data['Legs'][i]]))
Legs.reset_index(inplace=True)
Legs.head(1)

,index,Arrival,Carriers,Departure,DestinationStation,Directionality,Duration,FlightNumbers,Id,JourneyMode,OperatingCarriers,OriginStation,SegmentIds,Stops
0,0,2019-10-11T10:35:00,[1202],2019-10-11T08:40:00,13689,Outbound,175,"[{'FlightNumber': '3826', 'CarrierId': 1202}]",13870-1910110840--32221-0-13689-1910111035,Flight,[1202],13870,[26],[]


# **5. Carriers**

In [0]:
Carries = pd.DataFrame()
num_carries = len(json_data['Carriers'])
for i in range(num_carries):
  Carries = Carries.append(pd.DataFrame([json_data['Carriers'][i]]))
Carries.reset_index(inplace=True)
Carries.head(2)

,index,Code,DisplayCode,Id,ImageUrl,Name
0,0,I2,I2,1202,https://s1.apideeplink.com/images/airlines/I2.png,Iberia Express
1,0,IB,IB,1218,https://s1.apideeplink.com/images/airlines/IB.png,Iberia


#**6. Segments**

In [0]:
json_data['Segments'][0:2]

[{'ArrivalDateTime': '2019-10-11T10:30:00',
  'Carrier': 1218,
  'DepartureDateTime': '2019-10-11T08:40:00',
  'DestinationStation': 16864,
  'Directionality': 'Outbound',
  'Duration': 170,
  'FlightNumber': '3942',
  'Id': 0,
  'JourneyMode': 'Flight',
  'OperatingCarrier': 1202,
  'OriginStation': 13870},
 {'ArrivalDateTime': '2019-10-11T12:00:00',
  'Carrier': 1218,
  'DepartureDateTime': '2019-10-11T11:30:00',
  'DestinationStation': 13689,
  'Directionality': 'Outbound',
  'Duration': 30,
  'FlightNumber': '81',
  'Id': 1,
  'JourneyMode': 'Flight',
  'OperatingCarrier': 1612,
  'OriginStation': 16864}]

#7. Agents

In [0]:
Agents = pd.DataFrame()
num_agents = len(json_data['Agents'])
for i in range(num_agents):
  Agents = Agents.append(pd.DataFrame([json_data['Agents'][i]]))
Agents.reset_index(inplace=True)
Agents.head(2)

,index,Id,ImageUrl,Name,OptimisedForMobile,Status,Type
0,0,3390855,https://s1.apideeplink.com/images/websites/nor...,Norwegian,True,UpdatesComplete,Airline
1,0,4288780,https://s1.apideeplink.com/images/websites/vue...,Vueling Airlines,True,UpdatesComplete,Airline


# **Join all**

In [0]:
# partimos de los precios de las opciones que hay
# Cruzamos con las legs para saber que vuelos son (fecha,hora,aerolinea,duracion)

In [0]:
legs_aux = Legs[["Id","Departure","Arrival","Stops","Carriers","Duration"]]

In [0]:
tabla = pd.merge(Itineraries, legs_aux.add_prefix("Ida_"), how="left", left_on="OutboundLegId", right_on="Ida_Id")

In [0]:
tabla2 = pd.merge(tabla, legs_aux.add_prefix("Vuelta_"), how="left", left_on="InboundLegId", right_on="Vuelta_Id")

In [0]:
tabla2.loc[:, 'Agents'] = tabla2.Agents.map(lambda x: x[0])

In [0]:
tabla3 = pd.merge(tabla2, Agents[["Name","Id"]].rename(columns={"Name":"Billete_Vendido"}), how="left", left_on="Agents", right_on="Id").drop(columns="Id")

In [0]:
tabla3.loc[:, 'Ida_Carriers'] = tabla3.Ida_Carriers.map(lambda x: x[0])

In [0]:
tabla4 = pd.merge(tabla3, Carries[["Name","Id"]].rename(columns={"Name":"Ida_Aerolinea"}), how="left",
                  left_on="Ida_Carriers", right_on="Id").drop(columns="Id")

In [0]:
tabla4.loc[:, 'Vuelta_Carriers'] = tabla4.Vuelta_Carriers.map(lambda x: x[0])

In [0]:
tabla5 = pd.merge(tabla4, Carries[["Name","Id"]].rename(columns={"Name":"Vuelta_Aerolinea"}), how="left",
                  left_on="Vuelta_Carriers", right_on="Id").drop(columns="Id")

In [0]:
final = tabla5.copy()

In [0]:
final.head(2)

,index,Agents,DeeplinkUrl,Price,QuoteAgeInMinutes,InboundLegId,OutboundLegId,Ida_Id,Ida_Departure,Ida_Arrival,Ida_Stops,Ida_Carriers,Ida_Duration,Vuelta_Id,Vuelta_Departure,Vuelta_Arrival,Vuelta_Stops,Vuelta_Carriers,Vuelta_Duration,Billete_Vendido,Ida_Aerolinea,Vuelta_Aerolinea
0,0,4035342,http://partners.api.skyscanner.net/apiservices...,147.85,188,13689-1910131800--32680-0-13870-1910132150,13870-1910110705--32680-0-13689-1910110900,13870-1910110705--32680-0-13689-1910110900,2019-10-11T07:05:00,2019-10-11T09:00:00,[],1816,175,13689-1910131800--32680-0-13870-1910132150,2019-10-13T18:00:00,2019-10-13T21:50:00,[],1816,170,Travelgenio,Air Europa,Air Europa
1,1,3986805,http://partners.api.skyscanner.net/apiservices...,148.03,100,13689-1910131800--32680-0-13870-1910132150,13870-1910110705--32680-0-13689-1910110900,13870-1910110705--32680-0-13689-1910110900,2019-10-11T07:05:00,2019-10-11T09:00:00,[],1816,175,13689-1910131800--32680-0-13870-1910132150,2019-10-13T18:00:00,2019-10-13T21:50:00,[],1816,170,Travel2Be,Air Europa,Air Europa


In [0]:
final["Origen"] = "MAD"
final["Destino"] = "LPA"

In [0]:
final = final[["DeeplinkUrl","Origen","Destino","Ida_Departure","Ida_Arrival","Vuelta_Departure","Vuelta_Arrival","Price","Billete_Vendido","Ida_Aerolinea",
       "Vuelta_Aerolinea","Ida_Duration","Vuelta_Duration"]]

In [0]:
import datetime
final["FechaHora_Consulta"] = datetime.datetime.now()

In [0]:
final.to_csv("tabla_busqueda.csv")

In [0]:
final

,DeeplinkUrl,Origen,Destino,Ida_Departure,Ida_Arrival,Vuelta_Departure,Vuelta_Arrival,Price,Billete_Vendido,Ida_Aerolinea,Vuelta_Aerolinea,Ida_Duration,Vuelta_Duration,FechaHora_Consulta
0,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,147.85,Travelgenio,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
1,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,148.03,Travel2Be,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
2,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,151.98,Tripair,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
3,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,153.00,Budgetair.es,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
4,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,153.99,GotoGate,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
5,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,154.99,lastminute.com,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
6,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,154.99,Tix.es,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
7,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,159.78,Air Europa,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
8,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,162.60,eDreams,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799
9,http://partners.api.skyscanner.net/apiservices...,MAD,LPA,2019-10-11T07:05:00,2019-10-11T09:00:00,2019-10-13T18:00:00,2019-10-13T21:50:00,166.64,Opodo,Air Europa,Air Europa,175,170,2019-09-16 22:36:41.622799


In [0]:
# to do 
# finalizar mapeo variables Sí
# bucle hasta que acepte la página Sí
# bucle hasta que carge la página 0
# más de una página
# funcion
